# Network In Network的Pytorch实现

In [1]:
import time
import torch
from torch import nn, optim
import torchvision
from torchsummary import summary
import torch.nn.functional as F

## 定义模型

In [8]:
                    
class NiNModel(nn.Module):
    def __init__(self, classes):
        super(NiNModel, self).__init__()
        self.classes = classes
        self.nin_block1 = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=96, kernel_size=11, stride=4),
            nn.ReLU(),
            nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=96, out_channels=96, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2)
        )
        self.nin_block2 = nn.Sequential(
            nn.Conv2d(in_channels=96, out_channels=256, kernel_size=5, padding=2),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.nin_block3 = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=384, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=384, out_channels=384, kernel_size=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=2),
        )
        self.nin_block4 = nn.Sequential(
            nn.Conv2d(in_channels=384, out_channels=1024, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=1024, out_channels=1024, kernel_size=1),
            nn.ReLU(),
            nn.Conv2d(in_channels=1024, out_channels=self.classes, kernel_size=1),
            nn.ReLU()
        )
    def forward(self, x):
        x = self.nin_block1(x)
        x = self.nin_block2(x)
        x = self.nin_block3(x)
        x = self.nin_block4(x)
        #全局平均池化kernel_size为输入的高和宽
        x = F.avg_pool2d(x, kernel_size=x.shape[2:])
        output = x.view(x.shape[0], -1)
        return output

net = NiNModel(classes=10)
summary(net.to('cuda'), (3, 224, 224))
        
        
        
        

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 96, 54, 54]          34,944
              ReLU-2           [-1, 96, 54, 54]               0
            Conv2d-3           [-1, 96, 54, 54]           9,312
              ReLU-4           [-1, 96, 54, 54]               0
            Conv2d-5           [-1, 96, 54, 54]           9,312
              ReLU-6           [-1, 96, 54, 54]               0
         MaxPool2d-7           [-1, 96, 26, 26]               0
            Conv2d-8          [-1, 256, 26, 26]         614,656
              ReLU-9          [-1, 256, 26, 26]               0
           Conv2d-10          [-1, 256, 26, 26]          65,792
             ReLU-11          [-1, 256, 26, 26]               0
           Conv2d-12          [-1, 256, 26, 26]          65,792
             ReLU-13          [-1, 256, 26, 26]               0
        MaxPool2d-14          [-1, 256,